In [1]:
from pyspark import SparkContext
sc = SparkContext()

In [2]:
import math
import time

In [3]:
import winsound
def beep(n):
    for i in range(0,n):
        frequency = 500  # Set Frequency To 2500 Hertz
        duration = 500  # Set Duration To 1000 ms == 1 second
        winsound.Beep(frequency, duration)

In [4]:
# carregar base de dados
import os.path
fileName = os.path.join('C:\spark\Data', 'flights2015.csv')
numPartitions = 8
rawData = sc.textFile(fileName, numPartitions)


In [5]:
Cancelled = rawData.map(lambda x: x.split(",")[24])
featMonth = rawData.map(lambda x: x.split(",")[1])
featDayofMonth = rawData.map(lambda x: x.split(",")[2])
featDayofWeek = rawData.map(lambda x: x.split(",")[3])
featAirline =  rawData.map(lambda x: x.split(",")[4])

featOrigin =  rawData.map(lambda x: x.split(",")[7])
featDest =  rawData.map(lambda x: x.split(",")[8])
featTailNumber =  rawData.map(lambda x: x.split(",")[6])


In [6]:
def Entropia (classe):
    """Calcula a Entropia de Shannon de uma distribuição de dados.

    Args:
        classe (RDD): RDD contendo conjunto de dados a ser calculado a entropia.
                      Valores devem ser categóricos.

    Returns:
        float: valor de Entropia de Shannon calculado para o RDD.
    """
    #counts calcula paralelamente o conteúdo do RDD
    #como tuplas contendo (tipo, quantidade)
    counts = (classe.map(lambda x: (x, 1)).reduceByKey(lambda a,b: a + b))
    # n recebe o valor total de itens do RDD
    n = classe.count()
    
    # probs calcula a probabilidade de cada um dos estados do RDD
    probs = counts.map(lambda x: x[1]/float(n))
    
    # Entropia calcula a entropia do RDD
    ## a função map faz o calculo da Entropia de cada um dos estados
    ## a função reduce faz o somatório da entropia de Shannon
    entropia = (probs.map(lambda p: -p*math.log(p,2)).reduce(lambda a,b: a + b))
    
    # retorna valor escalar referênte a entropia do RDD.
    return entropia

 
def infoGain (feature, classe, H):
    """Calcula o ganho de informação de um atributo em relação a uma classe.

    Args:
        feature (RDD): RDD contendo os conjuntos de dados do atributo a ser
                       calculado o Ganho de Informação
        
        classe (RDD): RDD contendo conjunto de dados da classe
        
        H (float): Entropia da Classe, previamente calculada.

    Returns:
        float: valor de ganho de informação (redução da Entropia) que o atributo fornece sobre a classe
    """
    # calcula paralelamente o conteúdo do RDD
    # como tuplas contendo (tipo, quantidade)
    feat_count = feature.map(lambda x: (x, 1))\
                        .reduceByKey(lambda a,b: a + b)\
                        .collect()
    print feat_count
    print "\n"
 
    # calcula as Entropias de um conjunto da classe dado cada um dos estados do atributo   
    entropiasN = [Entropia(classe.zip(feature).filter(lambda x: x[1]==v).map(lambda x: x[0]))  for v,_ in feat_count]
    
    # calcula a quantidade de itens no atributo
    n = classe.count()
        
    # calcula o ganho de informação do atributo.
    ig = H - sum([(f[1]/float(n))*p for f,p in zip(feat_count, entropiasN)])
    
    return ig
    

In [7]:
start_time = time.time()
H =  Entropia (Cancelled)
print H

print("--- %s seconds ---" % (time.time() - start_time))
print "numer of partition:", numPartitions

0.115046214978
--- 113.213999987 seconds ---
numer of partition: 8


In [106]:
start_time = time.time()
igMonth = infoGain(featMonth, Cancelled, H)
print igMonth
print("--- %s seconds ---" % (time.time() - start_time))
print "numer of partition:", numPartitions

[(u'9', 464946), (u'11', 467972), (u'3', 504312), (u'5', 496993), (u'7', 520718), (u'10', 486165), (u'12', 479230), (u'1', 469968), (u'8', 510536), (u'4', 485151), (u'6', 503897), (u'2', 429191)]


0.00480131432616
--- 2258.47500014 seconds ---
numer of partition: 8


In [107]:
start_time = time.time()

igDayofMonth = infoGain(featDayofMonth, Cancelled, H)
print igDayofMonth
print("--- %s seconds ---" % (time.time() - start_time))
print "numer of partition:", numPartitions

[(u'11', 190756), (u'22', 192725), (u'3', 190007), (u'5', 189766), (u'7', 187598), (u'9', 194224), (u'10', 189288), (u'23', 193560), (u'13', 195089), (u'12', 190872), (u'1', 189477), (u'15', 192950), (u'14', 188611), (u'28', 191401), (u'17', 191319), (u'29', 179441), (u'16', 195899), (u'19', 193284), (u'18', 191393), (u'8', 193964), (u'4', 190893), (u'6', 191232), (u'24', 185017), (u'31', 103812), (u'25', 187317), (u'30', 178771), (u'26', 187387), (u'27', 191920), (u'2', 195986), (u'20', 195707), (u'21', 189413)]


0.00192616646485
--- 5671.07599998 seconds ---
numer of partition: 8


In [108]:
start_time = time.time()
igDayofWeek = infoGain(featDayofWeek, Cancelled, H)
print igDayofWeek
print("--- %s seconds ---" % (time.time() - start_time))
print "numer of partition:", numPartitions

[(u'3', 855897), (u'5', 862209), (u'7', 817764), (u'1', 865543), (u'4', 872521), (u'6', 700545), (u'2', 844600)]


0.000865229592909
--- 1307.579 seconds ---
numer of partition: 8


In [109]:
start_time = time.time()
igAirline = infoGain(featAirline, Cancelled, H)
print igAirline
print("--- %s seconds ---" % (time.time() - start_time))
print "numer of partition:", numPartitions

[(u'DL', 875881), (u'B6', 267048), (u'AA', 725984), (u'AS', 172521), (u'WN', 1261855), (u'NK', 117379), (u'HA', 76272), (u'UA', 515723), (u'OO', 588353), (u'MQ', 294632), (u'F9', 90836), (u'VX', 61903), (u'US', 198715), (u'EV', 571977)]


0.00430920158126
--- 2588.42599988 seconds ---
numer of partition: 8


In [110]:
beep(10)

In [8]:
start_time = time.time()
igOrigin = infoGain(featOrigin, Cancelled, H)
print igOrigin
print("--- %s seconds ---" % (time.time() - start_time))
print "numer of partition:", numPartitions

KeyboardInterrupt: 

In [111]:
start_time = time.time()
igDest = infoGain(featDest, Cancelled, H)
print igDest
print("--- %s seconds ---" % (time.time() - start_time))
print "numer of partition:", numPartitions

KeyboardInterrupt: 

In [11]:
# carregar base de dados
import os.path
fileName = os.path.join('C:\spark\Data', 'flights2015.csv')
numPartitions = 1
rawData = sc.textFile(fileName, numPartitions)


In [12]:
Cancelled = rawData.map(lambda x: x.split(",")[24])
featMonth = rawData.map(lambda x: x.split(",")[1])
featDayofMonth = rawData.map(lambda x: x.split(",")[2])
featDayofWeek = rawData.map(lambda x: x.split(",")[3])
featAirline =  rawData.map(lambda x: x.split(",")[4])

featOrigin =  rawData.map(lambda x: x.split(",")[7])
featDest =  rawData.map(lambda x: x.split(",")[8])
featTailNumber =  rawData.map(lambda x: x.split(",")[6])


In [13]:
start_time = time.time()
igMonth = infoGain(featMonth, Cancelled, H)
print igMonth
print("--- %s seconds ---" % (time.time() - start_time))
print "numer of partition:", numPartitions

[(u'9', 464946), (u'11', 467972), (u'3', 504312), (u'5', 496993), (u'7', 520718), (u'10', 486165), (u'12', 479230), (u'1', 469968), (u'8', 510536), (u'4', 485151), (u'6', 503897), (u'2', 429191)]


0.00480131432616
--- 2187.72099996 seconds ---
numer of partition: 1


In [14]:
start_time = time.time()

igDayofMonth = infoGain(featDayofMonth, Cancelled, H)
print igDayofMonth
print("--- %s seconds ---" % (time.time() - start_time))
print "numer of partition:", numPartitions

[(u'11', 190756), (u'22', 192725), (u'3', 190007), (u'5', 189766), (u'7', 187598), (u'9', 194224), (u'10', 189288), (u'23', 193560), (u'13', 195089), (u'12', 190872), (u'1', 189477), (u'15', 192950), (u'14', 188611), (u'28', 191401), (u'17', 191319), (u'29', 179441), (u'16', 195899), (u'19', 193284), (u'18', 191393), (u'8', 193964), (u'4', 190893), (u'6', 191232), (u'24', 185017), (u'31', 103812), (u'25', 187317), (u'30', 178771), (u'26', 187387), (u'27', 191920), (u'2', 195986), (u'20', 195707), (u'21', 189413)]


0.00192616646485
--- 5459.20799994 seconds ---
numer of partition: 1


In [15]:
start_time = time.time()
igDayofWeek = infoGain(featDayofWeek, Cancelled, H)
print igDayofWeek
print("--- %s seconds ---" % (time.time() - start_time))
print "numer of partition:", numPartitions

[(u'3', 855897), (u'5', 862209), (u'7', 817764), (u'1', 865543), (u'4', 872521), (u'6', 700545), (u'2', 844600)]


0.000865229592909
--- 1275.79400015 seconds ---
numer of partition: 1


In [16]:
start_time = time.time()
igAirline = infoGain(featAirline, Cancelled, H)
print igAirline
print("--- %s seconds ---" % (time.time() - start_time))
print "numer of partition:", numPartitions

[(u'DL', 875881), (u'B6', 267048), (u'AA', 725984), (u'AS', 172521), (u'WN', 1261855), (u'NK', 117379), (u'HA', 76272), (u'UA', 515723), (u'OO', 588353), (u'MQ', 294632), (u'F9', 90836), (u'VX', 61903), (u'US', 198715), (u'EV', 571977)]


0.00430920158126
--- 2516.27799988 seconds ---
numer of partition: 1


In [17]:
beep(10)

In [18]:
start_time = time.time()
igOrigin = infoGain(featOrigin, Cancelled, H)
print igOrigin
print("--- %s seconds ---" % (time.time() - start_time))
print "numer of partition:", numPartitions

[(u'10141', 71), (u'COS', 6881), (u'TWF', 815), (u'SGU', 1777), (u'14457', 236), (u'14307', 1098), (u'DRO', 2068), (u'11982', 371), (u'14952', 150), (u'DTW', 108500), (u'12402', 503), (u'11267', 753), (u'11603', 328), (u'11630', 177), (u'MDT', 3285), (u'10529', 1714), (u'MTJ', 1001), (u'10561', 183), (u'11111', 123), (u'MCO', 110982), (u'11274', 87), (u'12451', 1712), (u'OME', 663), (u'LGB', 8777), (u'10754', 81), (u'PUB', 264), (u'DAL', 59699), (u'OKC', 16144), (u'10631', 62), (u'12206', 287), (u'10800', 1768), (u'11618', 9622), (u'13541', 11), (u'10140', 1816), (u'15376', 1311), (u'SUN', 956), (u'ISP', 4392), (u'15389', 87), (u'14711', 62), (u'14487', 62), (u'12982', 881), (u'CMX', 668), (u'10577', 62), (u'12278', 698), (u'13873', 62), (u'12016', 31), (u'11503', 4), (u'SPI', 1587), (u'ACV', 1320), (u'12173', 3780), (u'13127', 53), (u'SGF', 6236), (u'14674', 93), (u'14027', 1742), (u'12953', 8590), (u'ABI', 2329), (u'BQK', 878), (u'CHO', 2204), (u'RDD', 727), (u'IMT', 620), (u'11057',

KeyboardInterrupt: 

In [ ]:
beep(10)

In [ ]:
start_time = time.time()
igDest = infoGain(featDest, Cancelled, H)
print igDest
print("--- %s seconds ---" % (time.time() - start_time))
print "numer of partition:", numPartitions

In [7]:
feat_count = Cancelled.map(lambda x: (x, 1))\
                        .reduceByKey(lambda a,b: a + b)\
                        .collect()

In [8]:
print feat_count


[(u'1', 89884), (u'0', 5729195)]
